In [2]:
!pip install langchain

In [3]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install pyautogen


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00


In [5]:
!pip install groq

In [ ]:
!pip install pyautogen groq pandas requests


In [ ]:
##########################Last###############################

In [ ]:
!pip install pyautogen langchain_groq requests



In [7]:
# ==============================
# ✅ 1) Setup
# ==============================

import os
import json
import requests
import pandas as pd
import re
from getpass import getpass
from hashlib import md5
from groq import Groq

# Set Groq API Key securely
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

# ==============================
# ✅ 2) Utilities
# ==============================

enrichment_cache = {}
aspect_cache = {}

def check_api_alive(url, timeout=3):
    try:
        r = requests.get(url, timeout=timeout)
        return r.status_code == 200
    except:
        return False

def enrich_prompt(requirements):
    cache_key = md5(json.dumps(requirements, sort_keys=True).encode()).hexdigest()
    if cache_key in enrichment_cache:
        return enrichment_cache[cache_key]

    prompt = f"""
Please rewrite these software requirements to be clearer and more complete:\n{json.dumps(requirements, indent=2)}
Return the improved version as a JSON object with the same keys.
"""
    response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )
    raw = response.choices[0].message.content.strip()
    try:
        improved = json.loads(raw)
    except:
        improved = requirements

    enrichment_cache[cache_key] = improved
    return improved

def extract_aspects(requirements):
    cache_key = md5(json.dumps(requirements, sort_keys=True).encode()).hexdigest()
    if cache_key in aspect_cache:
        return aspect_cache[cache_key]

    prompt = f"""
Given these software requirements:\n{json.dumps(requirements, indent=2)}
Extract the main functional aspects or features as a list of keywords for API matching.
Return the list as a valid JSON array.
"""
    response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    raw = response.choices[0].message.content.strip()
    try:
        aspects = json.loads(raw)
    except:
        match = re.search(r"\[.*?\]", raw, re.DOTALL)
        aspects = json.loads(match.group(0)) if match else []

    aspect_cache[cache_key] = aspects
    return aspects

def load_public_apis(limit=20):
    response = requests.get("https://api.apis.guru/v2/list.json")
    all_apis = response.json()
    services = []

    for i, (name, data) in enumerate(all_apis.items()):
        if i >= limit:
            break
        versions = data.get("versions", {})
        latest_version = list(versions.keys())[-1]
        api_info = versions[latest_version]

        try:
            spec_url = api_info.get("swaggerUrl")
            if spec_url and check_api_alive(spec_url):
                spec_response = requests.get(spec_url, timeout=5)
                if spec_response.status_code == 200:
                    spec = spec_response.json()
                    title = spec.get("info", {}).get("title", name)
                    description = spec.get("info", {}).get("description", "")

                    enhanced_features = []
                    for path, ops in spec.get("paths", {}).items():
                        for method, operation in ops.items():
                            feature = {
                                "path": path,
                                "method": method.upper(),
                                "summary": operation.get("summary", ""),
                                "operationId": operation.get("operationId", ""),
                                "tags": operation.get("tags", [])
                            }
                            enhanced_features.append(feature)

                    services.append({
                        "service_name": title,
                        "description": description,
                        "features": enhanced_features[:10]
                    })
        except:
            continue

    return services

def export_to_csv(json_result):
    try:
        data = json.loads(json_result)
        df = pd.DataFrame(data)
        df.to_csv("top_services.csv", index=False)
        print("✅ Exported to top_services.csv")
    except Exception as e:
        print(f"CSV export failed: {e}")

# ==============================
# ✅ 3) Manual User Input
# ==============================

questions = [
    "1️⃣ What is the primary goal of the software?",
    "2️⃣ What are the key features? (comma-separated)",
    "3️⃣ Who is the target audience?",
    "4️⃣ What platform should the software be on? (web, mobile, desktop, iOS, Android, etc.)",
    "5️⃣ What is the expected user load?",
    "6️⃣ Are there any budget constraints?",
    "7️⃣ Are there any timeline constraints?"
]

user_requirements = {}
for q in questions:
    user_requirements[q] = input(f"{q}\n")

print("\n📌 User Requirements:")
print(json.dumps(user_requirements, indent=2))

# ==============================
# ✅ 4) Run Discovery Pipeline
# ==============================

print("\n🔍 Enriching requirements...")
enriched = enrich_prompt(user_requirements)

print("\n🔑 Extracting aspects...")
aspects = extract_aspects(enriched)

print("\n🌐 Loading public APIs...")
services = load_public_apis(limit=30)

if not services:
    print("No public services could be loaded.")
else:
    system_prompt = f"""
You are an intelligent service matching agent.
Given the user's software requirements, extracted aspects, and a list of discovered API services (with their descriptions and enhanced features),
rank the top 5 that best match the user's needs.
Return a JSON array with: rank, service_name, matched_features, reason_for_match.
"""

    user_prompt = f"""
User Requirements:
{json.dumps(enriched, indent=2)}

Extracted Aspects:
{json.dumps(aspects, indent=2)}

Discovered Services:
{json.dumps(services, indent=2)}
"""

    response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=1500
    )

    result = response.choices[0].message.content.strip()
    try:
        json.loads(result)  # Validate
    except:
        print("⚠️ LLM response was not valid JSON. Output below:")
        print(result)
        result = None

    if result:
        with open("top_services.json", "w") as f:
            f.write(result)
        export_to_csv(result)
        print("\n🎯 Top Ranked Services:\n", result)



Enter your Groq API key: ··········
1️⃣ What is the primary goal of the software?
I want to build a transaction app for mobile users
2️⃣ What are the key features? (comma-separated)
money, balance, username
3️⃣ Who is the target audience?
user
4️⃣ What platform should the software be on? (web, mobile, desktop, iOS, Android, etc.)
mobile
5️⃣ What is the expected user load?
money
6️⃣ Are there any budget constraints?
yes
7️⃣ Are there any timeline constraints?
no

📌 User Requirements:
{
  "1\ufe0f\u20e3 What is the primary goal of the software?": "I want to build a transaction app for mobile users",
  "2\ufe0f\u20e3 What are the key features? (comma-separated)": "money, balance, username",
  "3\ufe0f\u20e3 Who is the target audience?": "user",
  "4\ufe0f\u20e3 What platform should the software be on? (web, mobile, desktop, iOS, Android, etc.)": "mobile",
  "5\ufe0f\u20e3 What is the expected user load?": "money",
  "6\ufe0f\u20e3 Are there any budget constraints?": "yes",
  "7\ufe0f\u20e

In [ ]:
#############################Last#####################################